Some label masks have one class missing. So need to check and figure out how to fix it.
Need the shape of `target` and `predicted` mask to be same for loss calcualtion.

In [2]:
# Import modules

import os, glob
import numpy as np
import torchio as tio

print('TorchIO version:', tio.__version__)

from platform import python_version
print(python_version())

TorchIO version: 0.18.76
3.9.7


In [16]:
dataset_dir = '/media/trojan/trojan/3D-CT-Artery-Segmentation/data_splitted'

In [17]:
images_dir = os.path.join(dataset_dir, 'train/volumes')
labels_dir = os.path.join(dataset_dir, 'train/masks')
image_paths = sorted(glob.glob(images_dir + '/' + '*.nii.gz'))
label_paths = sorted(glob.glob(labels_dir + '/' + '*.nii.gz'))
assert len(image_paths) == len(label_paths)

training_subjects = []
for (image_path, label_path) in zip(image_paths, label_paths):
    subject = tio.Subject(
        image=tio.ScalarImage(image_path),
        label=tio.LabelMap(label_path),
    )
    training_subjects.append(subject)
training_dataset = tio.SubjectsDataset(training_subjects)
print('Dataset size:', len(training_dataset), 'training subjects')

Dataset size: 612 training subjects


In [18]:
validation_images_dir = os.path.join(dataset_dir, 'val/volumes')
validation_labels_dir = os.path.join(dataset_dir, 'val/masks')
validation_image_paths = sorted(glob.glob(validation_images_dir + '/' + '*.nii.gz'))
validation_label_paths = sorted(glob.glob(validation_labels_dir + '/' + '*.nii.gz'))
assert len(validation_image_paths) == len(validation_label_paths)

validation_subjects = []
for (image_path, label_path) in zip(validation_image_paths, validation_label_paths):
    subject = tio.Subject(
        image=tio.ScalarImage(image_path),
        label=tio.LabelMap(label_path),
    )
    validation_subjects.append(subject)
validation_dataset = tio.SubjectsDataset(validation_subjects)
print('Dataset size:', len(validation_dataset), 'validation subjects')

Dataset size: 204 validation subjects


In [6]:
# count_equal = 0
# count_diff = 0
# for sample in validation_dataset:
#     transform = tio.OneHot()
#     transformed = transform(sample)
#     print(transformed.image.data.shape, transformed.label.data.shape)
#     if transformed.label.data.shape[0] == 3:
#         print("True")
#         count_equal += 1
#     else:
#         print("False")
#         count_diff += 1

Check all data. If size is different i.e. label is missing, then print the name of that volume.

In [7]:
# # Training data
# issue_masks = []

# for label_path in label_paths:
#     label = tio.LabelMap(label_path)
#     unique_labels = np.unique(label.data)
    
#     # Check label sizes and save the issue masks
#     if unique_labels.shape != (3,):
#         print("Size is not correct.")
#         print(label_path.split("/")[-1])
#         issue_masks.append(label_path.split("/")[-1])

In [8]:
# # Validation data
# val_issue_masks = []

# for label_path in validation_label_paths:
#     label = tio.LabelMap(label_path)
#     unique_labels = np.unique(label.data)
    
#     # Check label sizes and save the issue masks
#     if unique_labels.shape != (3,):
#         print("Size is not correct.")
#         print(label_path.split("/")[-1])
#         val_issue_masks.append(label_path.split("/")[-1])

Patient 2 and 6 have missing labels. Now we have to check which labels are missing for each patient.
label_map = {'Background': 0,'temporal_artery': 1,'facial_artery': 2}

In [9]:
# Check which labels are missing 

for label_path in glob.glob(validation_labels_dir + '/*.nii.gz'):
#     print(mask)
#     print(mask.split('/')[-1].split('.')[0].split('_')[0])
#     break
    split = label_path.split('/')[-1].split('.')[0].split('_')[0]
    
    if split == '2':
        print(f"*** Patient {split} ***")
        label = tio.LabelMap(label_path)
        unique_labels = np.unique(label.data)
        print(unique_labels)
    
    if split == '6':
        print(f"*** Patient {split} ***")
        label = tio.LabelMap(label_path)
        unique_labels = np.unique(label.data)
        print(unique_labels)
    

    

Patient 2 has a missing label and Patient 6 has an extra label.

# Extra class fix

The `extra` label issue is because of separate label for left and right **facial artery**. I can fix it by combining them into one label.
However I need to do it before apply `transform.Compose()`.

In [5]:
# sample_path = os.path.join(labels_dir, '6.nii.gz')
# label = tio.LabelMap(sample_path)
# unique_labels = np.unique(label.data)
# print(unique_labels)

[0 1 2 3]


In [9]:
# label.data[label.data==3] = 2
# label.data = np.where(label.data > 2, 2, label.data)

/tmp/ipykernel_337456/4153960674.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) data. (Setting the image data with the property setter is deprecated. Use the set_data() method instead) -- Deprecated since version 0.18.16.
  label.data = np.where(label.data > 2, 2, label.data)


In [10]:
# unique_labels = np.unique(label.data)
# print(unique_labels)

[0 1 2]


In [11]:
# images_dir = os.path.join(dataset_dir, 'train/volumes')
# labels_dir = os.path.join(dataset_dir, 'train/masks')
# image_paths = sorted(glob.glob(images_dir + '/' + '*.nii.gz'))
# label_paths = sorted(glob.glob(labels_dir + '/' + '*.nii.gz'))
# assert len(image_paths) == len(label_paths)

# training_subjects_fixed = []
# for (image_path, label_path) in zip(image_paths, label_paths):
#     label = tio.LabelMap(label_path)
#     unique_labels = np.unique(label.data)
    
#     if len(unique_labels) > 3: 
#         print(label_path)
#     label.data[label.data==3] = 2
    
#     subject = tio.Subject(
#         image=tio.ScalarImage(image_path),
#         label=label
#     )
        
#     training_subjects_fixed.append(subject)

# # print('Dataset size:', len(training_subjects_fixed), 'training subjects')

KeyboardInterrupt: 

Online is taking really long time. Need to do it offline. Change the values and save the mask again.

In [27]:
# Save all paths

issue_file_paths = []

for label_path in glob.glob(labels_dir + '/*.nii.gz'):
    split = label_path.split('/')[-1].split('.')[0].split('_')[0]
    
    if split == '6':
        issue_file_paths.append(label_path)

## Test on one sample volume

In [30]:
# for path in issue_file_paths:
#     label = tio.LabelMap(sample_path)
#     unique_labels = np.unique(label.data)
#     print(unique_labels)
#     label.data[label.data==3] = 2
#     unique_labels = np.unique(label.data)
#     print(unique_labels)
#     break

[0 1 2 3]
[0 1 2]


In [33]:
for path in issue_file_paths:
    label = tio.LabelMap(sample_path)
    label.data[label.data==3] = 2
    label.save("./temp/test.nii.gz")
    break

In [34]:
# Test the output
path = "./temp/test.nii.gz"
label = tio.LabelMap(path)
unique_labels = np.unique(label)
print(unique_labels)

[0 1 2]


## Apply on training data

In [38]:
for path in issue_file_paths:
    name = path.split("/")[-1]
    label = tio.LabelMap(sample_path)
    label.data[label.data==3] = 2
    label.save(f"./temp/train/{name}")

## Apply on validation data

In [39]:
validation_issue_file_paths = []

for label_path in glob.glob(validation_labels_dir + '/*.nii.gz'):
    split = label_path.split('/')[-1].split('.')[0].split('_')[0]
    
    if split == '6':
        validation_issue_file_paths.append(label_path)

In [40]:
for path in validation_issue_file_paths:
    name = path.split("/")[-1]
    label = tio.LabelMap(sample_path)
    label.data[label.data==3] = 2
    label.save(f"./temp/val/{name}")

## Check Shapes

In [5]:
train_fixed = "./temp/train"
for subject in glob.glob(train_fixed + "/*.nii.gz"):
    label = tio.LabelMap(subject)
    unique_labels = np.unique(label.data)
    print(unique_labels)

[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]


In [6]:
validation_fixed = "./temp/val"
for subject in glob.glob(validation_fixed + "/*.nii.gz"):
    label = tio.LabelMap(subject)
    unique_labels = np.unique(label.data)
    print(unique_labels)

[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]


Patient 2 issue i.e. missing class might be related to `loss function`.
Need to check it more.